In [1]:
# read data from csv file
import pandas as pd
import numpy as np
    
# read data from csv file
df = pd.read_csv('data.csv')

In [2]:
# For the search bar if the search bar doesn't work remove it
# Get the unique values of Titular
titular = df['Titular'].unique()
titular_list = list(titular)

In [3]:
# Take a 100 sample of df
df_sample = df.sample(100)

__Preprocessing__:

For a 100 sample takes 50 seconds

In [4]:
#pip install geopy
from geopy.geocoders import Nominatim

# Get the coordinates of a city (Municipio)

def get_coordinates(city):

    geolocator = Nominatim(user_agent="spain_explorer")
    location = geolocator.geocode(city) 
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

# Create a new dataframe with the coordinates

df_sample['Latitude'], df_sample['Longitude'] = zip(*df_sample.apply(lambda x: get_coordinates(x['Provincia_FREQ']), axis=1))



In [9]:
l = df['Provincia_FREQ'].unique().tolist()
for pro in l:
    print(get_coordinates(pro))

(42.844072249999996, -2.6820829322126283)
(43.238482000000005, -2.85163163999482)
(43.144527800000006, -2.2037955488327494)
(42.2814642, -2.482804975070473)
(42.61254875, -1.830787673361753)
(42.343926, -3.696977)
(43.1595664, -4.0878382)
(40.4167047, -3.7035825)
(41.6521342, -0.8809428)
(41.3828939, 2.1774322)
(41.9793006, 2.8199439)
(41.6147605, 0.6267842)
(41.1172364, 1.2546057)
(39.9881471, -0.051915)
(42.13606145, -0.029802662719165485)
(39.613432, 2.8829026938823548)
(43.3709703, -8.3959425)
(43.0462247, -7.4739921)
(42.1968459, -7.6114105)
(42.6075172, -8.4714942)
(42.63414505, -5.971415104539984)
(43.3133868, -5.94192)
(41.6857693, -5.942315044623113)
(36.8414197, -2.4628135)
(36.5297438, -6.2928976)
(37.9923795, -1.1305431)
(37.1734995, -3.5995337)
(37.3886303, -5.9953403)
(38.88964355, -6.980142450723942)
(36.7213028, -4.4216366)
(37.8845813, -4.7760138)
(37.2575874, -6.9484945)
(37.9557275, -3.492055612037218)
(35.29291145, -2.950720236037561)
(38.3436365, -0.4881708)
(39.33

In [5]:
# Municipio_Count is the number of radiolinks per Municipio

# Create a column 'Num_Radiolinks' 

df_sample['Num_Radiolinks'] = 1

# Group the rows of the dataframe by the 'Municipio' column and sum the 'Num_Radiolinks' column
df_agg = df_sample.groupby(['Provincia_FREQ'])['Num_Radiolinks'].sum().reset_index()

# Rename the column 'Num_Radiolinks' to the desired name
df_agg.rename(columns={'Num_Radiolinks': 'Municipio_Count'}, inplace=True)

# add the 'Municipio_Count' from df_agg to the df_sample joining by 'Municipio'

df_sample = pd.merge(df_sample, df_agg, on='Provincia_FREQ')

__Map__


In [6]:
import folium
import pandas as pd

from folium.plugins import FastMarkerCluster
from folium import plugins

# Create a map centered on Spain
#spain_map = folium.Map(location=[40.416775, -3.703790], zoom_start=6)
spain_map = spain_map = folium.Map(location=[40.416775, -3.703790], zoom_start=6, tiles="Stamen Toner", control_scale=False)


# Create a linear color map for the number of Radiolinks per Municipio, with red for low values and green for high values
color_map = folium.LinearColormap(
    colors=['red', 'yellow', 'green'],
    vmin=df_sample['Municipio_Count'].min(),
    vmax=df_sample['Municipio_Count'].max()
)

# Create a layer for municipalities with the number of Radiolinks, and add it to the map
municipalities = folium.FeatureGroup(name='Municipalities')
for index, row in df_sample.iterrows():
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Municipality: {row['Provincia_FREQ']}<br>Number of Radiolinks: {row['Municipio_Count']}",
        radius=15000,
        fill=True,
        stroke=False,
        color=color_map(row['Municipio_Count']),
        fill_opacity=0.5
    ).add_to(municipalities)

municipalities.add_to(spain_map)

# Add the color map to the map
color_map.add_to(spain_map)

marker_cluster = FastMarkerCluster(
    data=df_sample[['Latitude', 'Longitude']].values.tolist(),
    popup=df_sample.apply(lambda x: f"Municipality: {x['Provincia_FREQ']}<br>Number of Radiolinks: {x['Municipio_Count']}", axis=1).tolist()
).add_to(spain_map)


# If the search bar doesn't work remove it or leave it to flex, the code works fine 
# Create the search bar
search_bar = plugins.Search(
    layer=municipalities,
    geom_type='Point',
    placeholder="Search for a Company",
    collapsed=False,
    search_label='Titular',
    search_field='Titular'
).add_to(spain_map)

# Add the autocomplete feature using jquery-ui
spain_map.get_root().header.add_child(
    folium.JavascriptLink('https://code.jquery.com/ui/1.12.1/jquery-ui.js'),
    name='jquery'
)

spain_map.get_root().header.add_child(
    folium.Element(
        '<style>\n'
        '.ui-autocomplete-loading {\n'
        '  background: white url("https://jqueryui.com/resources/demos/autocomplete/images/ui-anim_basic_16x16.gif") right center no-repeat;\n'
        '}\n'
        '.ui-autocomplete {\n'
        '  max-height: 200px;\n'
        '  overflow-y: auto;\n'
        '}\n'
        '</style>'
    )
)

spain_map.get_root().script.add_child(
    folium.Element(
        f'''
        $(document).ready(function() {{
            var searchData = {titular_list};
            $("#search").autocomplete({{
                source: searchData
            }});  
        }});
        '''
    )
)

# show the map

spain_map